In [1]:
import h5py
from collections import defaultdict
import random
import numpy as np
import pickle
import os

In [2]:
file_path = "../data/merge.hdf5"


hdf5_file = h5py.File(file_path, 'r')
data_group = hdf5_file['data']

In [3]:
sample_key = list(data_group.keys())[0]
sample = data_group[sample_key]

print("Attributes in sample : ",dict(sample.attrs))
print("Key : ",sample_key)

Attributes in sample :  {'back_azimuth_deg': 159.3, 'coda_end_sample': array([[2896.]], dtype=float32), 'network_code': 'TA', 'p_arrival_sample': 700.0, 'p_status': 'manual', 'p_travel_sec': 17.079999923706055, 'p_weight': 0.5, 'receiver_code': '109C', 'receiver_elevation_m': 150.0, 'receiver_latitude': 32.8889, 'receiver_longitude': -117.1051, 'receiver_type': 'BH', 's_arrival_sample': 1894.0, 's_status': 'manual', 's_weight': 0.5, 'snr_db': array([56.79999924, 55.40000153, 47.40000153]), 'source_depth_km': 0.45, 'source_depth_uncertainty_km': 'None', 'source_distance_deg': 0.92, 'source_distance_km': 102.09, 'source_error_sec': 1.1119, 'source_gap_deg': 107.466, 'source_horizontal_uncertainty_km': 4.6403, 'source_id': '8556349', 'source_latitude': 33.7496, 'source_longitude': -117.4938, 'source_magnitude': 3.6, 'source_magnitude_author': 'None', 'source_magnitude_type': 'ml', 'source_mechanism_strike_dip_rake': 'None', 'source_origin_time': '2006-07-23 15:58:50.88', 'source_origin_un

In [4]:
attrs = sample.attrs
print("Label:", attrs['trace_category'])
print("Reciever code : ",attrs['receiver_code'])

Label: earthquake_local
Reciever code :  109C


In [5]:
earthquake_keys = np.load("../preprocessed/earthquake_keys.npy", allow_pickle=True)
noise_keys = np.load("../preprocessed/noise_keys.npy", allow_pickle=True)
subset_keys = np.concatenate([earthquake_keys, noise_keys])

print(f"Total subset keys: {len(subset_keys)}")

Total subset keys: 70000


In [ ]:
key_to_label = {}
station_to_keys = defaultdict(list)

for key in subset_keys:
    attrs = data_group[key].attrs
    label = 1 if attrs["trace_category"] == "earthquake_local" else 0
    station = attrs["receiver_code"]
    key_to_label[key] = label
    station_to_keys[station].append(key)

print(f"Total stations in subset: {len(station_to_keys)}")

with open("../preprocessed/key_to_label.pkl", "wb") as f:
    pickle.dump(key_to_label, f)

with open("../preprocessed/station_to_keys.pkl", "wb") as f:
    pickle.dump(station_to_keys, f)

In [7]:
with open("../preprocessed/key_to_label.pkl", "rb") as f:
    key_to_label = pickle.load(f)

with open("../preprocessed/station_to_keys.pkl", "rb") as f:
    station_to_keys = pickle.load(f)

In [8]:
stations = list(station_to_keys.keys())
random.shuffle(stations)

n_total = len(stations)
n_train = int(0.8 * n_total)
n_val = int(0.1 * n_total)

train_stations = stations[:n_train]
val_stations = stations[n_train:n_train + n_val]
test_stations = stations[n_train + n_val:]

In [9]:
def collect_samples(station_list):
  return np.array([
    (key, key_to_label[key])
    for st in station_list
    for key in station_to_keys[st]
  ], dtype=object)

train_samples = collect_samples(train_stations)
val_samples   = collect_samples(val_stations)
test_samples  = collect_samples(test_stations)

In [ ]:
np.save("../preprocessed/train_split.npy", train_samples)
np.save("../preprocessed/val_split.npy", val_samples)
np.save("../preprocessed/test_split.npy", test_samples)

In [15]:
from scipy.signal import stft
from tqdm import tqdm


os.makedirs("../preprocessed/spectrograms", exist_ok=True)

# STFT parameters
fs = 100  # Hz (STEAD sampling rate)
nperseg = 128
noverlap = 64
nfft = 128

def preprocess_sample(hdf5_file, key):
    sample = hdf5_file['data'][key][:]
    sample = (sample - sample.mean(axis=0)) / (sample.std(axis=0) + 1e-6)

    specs = []
    for ch in range(3):
        f, t, Zxx = stft(sample[:, ch], fs=fs, nperseg=nperseg,
                         noverlap=noverlap, nfft=nfft)
        Sxx = np.abs(Zxx)
        Sxx = (Sxx - Sxx.min()) / (Sxx.max() - Sxx.min() + 1e-6)
        specs.append(Sxx)

    return np.stack(specs, axis=-1).astype(np.float32)

def preprocess_split(split_name):
    samples = np.load(f"../preprocessed/{split_name}_split.npy", allow_pickle=True)
    X_list, y_list = [], []
    with h5py.File(file_path, "r") as f:
        for i, (key, label) in enumerate(tqdm(samples, desc=f"{split_name} processing")):
            spec = preprocess_sample(f, key)
            X_list.append(spec)
            y_list.append(label)

    X = np.array(X_list)
    y = np.array(y_list)
    np.save(f"../preprocessed/spectrograms/X_{split_name}.npy", X)
    np.save(f"../preprocessed/spectrograms/y_{split_name}.npy", y)
    print(f"{split_name} saved: {X.shape}, {y.shape}")


preprocess_split("train")
preprocess_split("val")
preprocess_split("test")


train processing: 100%|██████████| 57789/57789 [20:39<00:00, 46.64it/s]


train saved: (57789, 65, 95, 3), (57789,)


val processing: 100%|██████████| 7232/7232 [02:27<00:00, 48.97it/s]


val saved: (7232, 65, 95, 3), (7232,)


test processing: 100%|██████████| 4979/4979 [01:46<00:00, 46.81it/s]


test saved: (4979, 65, 95, 3), (4979,)


In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix

In [20]:
X_train = np.load("../preprocessed/spectrograms/X_train.npy")
y_train = np.load("../preprocessed/spectrograms/y_train.npy")
X_val = np.load("../preprocessed/spectrograms/X_val.npy")
y_val = np.load("../preprocessed/spectrograms/y_val.npy")
X_test = np.load("../preprocessed/spectrograms/X_test.npy")
y_test = np.load("../preprocessed/spectrograms/y_test.npy")

In [21]:
print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

Train: (57789, 65, 95, 3), Val: (7232, 65, 95, 3), Test: (4979, 65, 95, 3)


In [19]:
data_augmentation = tf.keras.Sequential([
    layers.RandomTranslation(height_factor=0.03, width_factor=0.05),
    layers.RandomZoom(0.05),
    layers.RandomContrast(0.2),
    layers.Lambda(lambda x: x + tf.random.normal(tf.shape(x), stddev=0.02))
])

In [22]:
BATCH_SIZE = 32

def make_dataset(X,y,training=False):
  dataset = tf.data.Dataset.from_tensor_slices((X,y))
  if training:
    dataset = dataset.shuffle(buffer_size=2000)
    dataset = dataset.map(lambda x,y : (data_augmentation(x,training=True),y),
                          num_parallel_calls=tf.data.AUTOTUNE)
  dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
  return dataset

In [23]:
train_ds = make_dataset(X_train, y_train, training=True)
val_ds = make_dataset(X_val, y_val)
test_ds = make_dataset(X_test, y_test)

In [24]:
for batch_x, batch_y in train_ds.take(1):
  print("Batch X shape:", batch_x.shape)
  print("Batch Y shape:", batch_y.shape)
  break

Batch X shape: (32, 65, 95, 3)
Batch Y shape: (32,)


In [34]:
def build_cnn(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),

        layers.Conv2D(16, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),

        layers.Dense(64, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

model = build_cnn(X_train.shape[1:])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 65, 95, 16)        448       
                                                                 
 batch_normalization_9 (Batc  (None, 65, 95, 16)       64        
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 47, 16)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 32, 47, 32)        4640      
                                                                 
 batch_normalization_10 (Bat  (None, 32, 47, 32)       128       
 chNormalization)                                                
                                                      

In [35]:
from tensorflow.keras import callbacks

early_stop = callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

In [36]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,               
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/25
1806/1806 [==============================] - 36s 19ms/step - loss: 0.1134 - accuracy: 0.9623 - val_loss: 0.3192 - val_accuracy: 0.8630
Epoch 2/25
1806/1806 [==============================] - 40s 22ms/step - loss: 0.0574 - accuracy: 0.9821 - val_loss: 0.7476 - val_accuracy: 0.7045
Epoch 3/25
1806/1806 [==============================] - 37s 20ms/step - loss: 0.0418 - accuracy: 0.9870 - val_loss: 1.1333 - val_accuracy: 0.6847
Epoch 4/25
1806/1806 [==============================] - 37s 20ms/step - loss: 0.0333 - accuracy: 0.9899 - val_loss: 0.6961 - val_accuracy: 0.8204
Epoch 5/25
1806/1806 [==============================] - 37s 20ms/step - loss: 0.0296 - accuracy: 0.9910 - val_loss: 0.1592 - val_accuracy: 0.9403
Epoch 6/25
1806/1806 [==============================] - 44s 24ms/step - loss: 0.0261 - accuracy: 0.9920 - val_loss: 0.1906 - val_accuracy: 0.9257
Epoch 7/25
1806/1806 [==============================] - 42s 23ms/step - loss: 0.0218 - accuracy: 0.9934 - val_loss: 0.0766 -

In [37]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

156/156 [==============================] - 1s 7ms/step - loss: 0.0103 - accuracy: 0.9966
Test Loss: 0.0103
Test Accuracy: 0.9966


In [38]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix


y_pred_probs = model.predict(test_ds)
y_pred = (y_pred_probs > 0.5).astype(int)


y_true = np.concatenate([y for x, y in test_ds], axis=0)


print(classification_report(y_true, y_pred, target_names=["Noise","Earthquake"]))


cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)


156/156 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

       Noise       0.99      1.00      1.00      2145
  Earthquake       1.00      0.99      1.00      2834

    accuracy                           1.00      4979
   macro avg       1.00      1.00      1.00      4979
weighted avg       1.00      1.00      1.00      4979

Confusion Matrix:
 [[2144    1]
 [  16 2818]]
